In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pickle 

In [2]:
dir = '/content/drive/MyDrive/20212/THDL/data/after_matching/data_final.csv'
save_dir = '/content/drive/MyDrive/20212/THDL/data/after_matching/suggesting_hotels.csv'
save_model = '/content/drive/MyDrive/20212/THDL/'

# **Đọc dữ liệu**

In [3]:
df = pd.read_csv(dir)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7060 entries, 0 to 7059
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Image              7056 non-null   object 
 1   new_name           7054 non-null   object 
 2   new_add            7057 non-null   object 
 3   ID                 7060 non-null   int64  
 4   Cluster            572 non-null    float64
 5   Website            7060 non-null   object 
 6   City               7060 non-null   object 
 7   Hotel name         7060 non-null   object 
 8   Url                7060 non-null   object 
 9   Address            7060 non-null   object 
 10  Stars              7060 non-null   float64
 11  Price              7060 non-null   int64  
 12  Rating             7060 non-null   float64
 13  Number of reviews  7060 non-null   int64  
 14  Reviews            4735 non-null   object 
 15  Facilities         6997 non-null   object 
 16  Description        6291 

In [5]:
hotels_df = df[['Image', 'City', 'Hotel name', 'Url',
                'Address', 'Stars', 'Price', 'Rating',
                'Number of reviews', 'Reviews', 'Facilities', 
                'Description', 'Nearby places']]

In [6]:
hotels_df = hotels_df.reset_index()
hotels_df.rename({'index':'Hotel_id'}, axis='columns', inplace=True)

In [7]:
cols = ['Hotel_id',  'Price', 'Rating', 'Number of reviews']
tmp_df = hotels_df[cols]

In [8]:
tmp_df.head()

,Hotel_id,Price,Rating,Number of reviews
0,0,421890,7.8,86
1,1,1316000,9.3,209
2,2,625000,9.6,217
3,3,1800000,9.1,72
4,4,1262250,9.4,627


# **Tìm KS có độ tương đồng cao nhất**

In [9]:
# ball_knn = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(tmp_df.set_index('Hotel_id'))
# # Its important to use binary mode 
# knnPickle = open(save_model+'ballknn_file', 'wb') 

In [10]:
# kd_knn = NearestNeighbors(n_neighbors=6, algorithm='kd_tree').fit(tmp_df.set_index('Hotel_id'))
# # Its important to use binary mode 
# knnPickle_1 = open(save_model+'kdknn_file', 'wb') 

In [11]:
auto_knn = NearestNeighbors(n_neighbors=6, algorithm='auto').fit(tmp_df.set_index('Hotel_id'))
knnPickle_2 = open(save_model+'autoknn_file', 'wb') 

In [12]:
# source, destination 
# pickle.dump(ball_knn, knnPickle) 
# pickle.dump(kd_knn, knnPickle_1) 
pickle.dump(auto_knn, knnPickle_2) 

In [13]:
# load the model from disk
loaded_model_auto = pickle.load(open(save_model+'autoknn_file', 'rb'))
# loaded_model_ball = pickle.load(open(save_model+'ballknn_file', 'rb'))
# loaded_model_kd = pickle.load(open(save_model+'kdknn_file', 'rb'))

In [14]:
tmp_df.set_index('Hotel_id').values[0].reshape(1,-1)

array([[4.2189e+05, 7.8000e+00, 8.6000e+01]])

In [15]:
def similarHotels(hotelID = 1):
  info = tmp_df[tmp_df['Hotel_id'] == hotelID].index[0]
  test = tmp_df.set_index('Hotel_id').values[info].reshape(1,-1)
  res = loaded_model_auto.kneighbors(test, return_distance=False)
  return res

In [16]:
hotelID = 5
result = similarHotels(hotelID)[0][1:]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


In [17]:
print('Khách sạn đang xem')
hotels_df[hotels_df['Hotel_id'] == hotelID][['Hotel_id','Hotel name']]

Khách sạn đang xem


,Hotel_id,Hotel name
5,5,Căn hộ Mới Sumitomo Hotel & Service Apartment ...


In [18]:
print('Các khách sạn tương đồng')
hotels_df[hotels_df['Hotel_id'].isin(result)][['Hotel_id','Hotel name']]

Các khách sạn tương đồng


,Hotel_id,Hotel name
1711,1711,Khách sạn Hotel L'Odéon Phu My Hung
2391,2391,Khách sạn Mới Ami Le Jaune DaLat Homestay
2538,2538,Căn hộ Mới Muong Thanh Seaview Apartment Da Nang
5467,5467,Common Inn
5816,5816,Lan Anh Dalat Hotel


# **Danh sách KS tương đồng nhau**

In [19]:
list_hotel = hotels_df['Hotel_id'].values

suggest = np.zeros(shape=(6, len(list_hotel)), dtype=int)
for i in range(1, 6):
  for j in range(len(list_hotel)):
    suggest[i][j] = similarHotels(list_hotel[j])[0][i]


suggested = {
    'Hotel_id': list_hotel,
    'suggest1': suggest[1],
    'suggest2': suggest[2],
    'suggest3': suggest[3],
    'suggest4': suggest[4],
    'suggest5': suggest[5]
}

df2 = pd.DataFrame(suggested)
df2.sort_values(by='Hotel_id', ascending=True, inplace=True)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but Neare

In [20]:
df2

,Hotel_id,suggest1,suggest2,suggest3,suggest4,suggest5
0,0,4137,6943,5671,6843,6755
1,1,4012,1020,6994,1199,5543
2,2,2865,2522,6122,5651,3180
3,3,5611,1729,819,1344,2566
4,4,4844,6042,2893,6019,6819
...,...,...,...,...,...,...
7055,7055,3119,1001,5481,1653,5920
7056,7056,5325,844,4903,2962,2957
7057,7057,2653,6947,3197,6069,6094
7058,7058,1007,668,230,2650,2733


# **Lưu file suggest**

In [21]:
df2.to_csv(save_dir, index=False)